## 1. Import


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


## 2. 평가 함수 정의 (NMAE 집중)


In [2]:
def comovement_nmae(answer_df, submission_df, eps=1e-6):
    """
    전체 U = G ∪ P에 대한 clipped NMAE 계산
    NMAE가 낮을수록 좋음 (0에 가까울수록 좋음)
    """
    ans = answer_df[["leading_item_id", "following_item_id", "value"]].copy()
    sub = submission_df[["leading_item_id", "following_item_id", "value"]].copy()
    ans["pair"] = list(zip(ans["leading_item_id"], ans["following_item_id"]))
    sub["pair"] = list(zip(sub["leading_item_id"], sub["following_item_id"]))
    
    G = set(ans["pair"])
    P = set(sub["pair"])
    U = G | P
    
    ans_val = dict(zip(ans["pair"], ans["value"]))
    sub_val = dict(zip(sub["pair"], sub["value"]))
    
    errors = []
    for pair in U:
        if pair in G and pair in P:
            # 정수 변환(반올림)
            y_true = int(round(float(ans_val[pair])))
            y_pred = int(round(float(sub_val[pair])))
            rel_err = abs(y_true - y_pred) / (abs(y_true) + eps)
            rel_err = min(rel_err, 1.0)  # 오차 100% 이상은 100%로 간주
        else:
            rel_err = 1.0  # FN, FP는 오차 100%
        errors.append(rel_err)
    
    return np.mean(errors) if errors else 1.0


## 3. 데이터 전처리 및 학습/검증 분리


In [3]:
train = pd.read_csv('../data/raw/train.csv')

# year, month, item_id 기준으로 value 합산
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

# 2025-07-01을 기준으로 학습/검증 분리
val_date = pd.to_datetime("2025-07-01")

# 학습 데이터: 2025-07-01 이전 데이터만 사용
pivot_train = pivot.loc[:, pivot.columns < val_date].copy()
print(f"학습 데이터 기간: {pivot_train.columns.min()} ~ {pivot_train.columns.max()}")
print(f"학습 데이터 shape: {pivot_train.shape}")

# 검증 데이터: 2025-07-01 데이터
if val_date in pivot.columns:
    print(f"\n검증 데이터 날짜: {val_date}")
    print(f"검증 데이터 shape: ({pivot.shape[0]}, 1)")
else:
    print(f"\n경고: {val_date} 데이터가 pivot에 없습니다.")


학습 데이터 기간: 2022-01-01 00:00:00 ~ 2025-06-01 00:00:00
학습 데이터 shape: (100, 42)

검증 데이터 날짜: 2025-07-01 00:00:00
검증 데이터 shape: (100, 1)


## 4. 공행성쌍 탐색 (학습 데이터만 사용)


In [4]:
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(
    pivot, 
    max_lag=6, 
    min_nonzero=12, 
    corr_threshold=0.4
):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for i, leader in tqdm(enumerate(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)
    return pairs

# 학습 데이터로만 공행성쌍 탐색
pairs = find_comovement_pairs(pivot_train)
print("탐색된 공행성쌍 수:", len(pairs))
pairs.head()


100it [00:09, 10.09it/s]

탐색된 공행성쌍 수: 1453


,leading_item_id,following_item_id,best_lag,max_corr
0,AANGBULD,APQGTRMF,5,-0.459240
1,AANGBULD,DEWLVASR,6,0.673163
2,AANGBULD,DNMPSKTB,4,-0.434721
3,AANGBULD,EVBVXETX,6,0.453442
4,AANGBULD,FTSVTTSR,3,0.533976


## 5. 고급 Feature Engineering 및 학습 데이터 생성


In [5]:
def build_training_data_enhanced(pivot, pairs, use_log_transform=False):
    """
    향상된 Feature Engineering을 적용한 학습 데이터 생성
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 2), n_months - 1):  # 최소 2개월 전 데이터 필요
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            b_t_2 = b_series[t - 2] if t >= 2 else 0.0
            a_t_lag = a_series[t - lag]
            a_t_lag_1 = a_series[t - lag - 1] if t - lag - 1 >= 0 else 0.0
            b_t_plus_1 = b_series[t + 1]

            # 기본 feature
            features = {
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
            }
            
            # 추가 feature: 이동평균
            window = min(3, t + 1)
            features["b_t_ma3"] = np.mean(b_series[max(0, t - window + 1):t + 1])
            features["a_t_lag_ma3"] = np.mean(a_series[max(0, t - lag - window + 1):t - lag + 1]) if t - lag >= 0 else 0.0
            
            # 추가 feature: 트렌드 (변화율)
            features["b_trend"] = (b_t - b_t_1) / (b_t_1 + 1e-6) if b_t_1 > 0 else 0.0
            features["b_trend_2"] = (b_t_1 - b_t_2) / (b_t_2 + 1e-6) if b_t_2 > 0 else 0.0
            features["a_trend"] = (a_t_lag - a_t_lag_1) / (a_t_lag_1 + 1e-6) if a_t_lag_1 > 0 else 0.0
            
            # 추가 feature: 정규화된 값
            b_mean = np.mean(b_series[:t+1])
            a_mean = np.mean(a_series[:t-lag+1]) if t - lag >= 0 else 1.0
            features["b_t_scaled"] = b_t / (b_mean + 1e-6)
            features["a_t_lag_scaled"] = a_t_lag / (a_mean + 1e-6)
            
            # 추가 feature: 상관관계 가중치
            features["a_t_lag_weighted"] = a_t_lag * abs(corr)
            
            # 추가 feature: lag별 특성
            features["lag_1"] = 1.0 if lag == 1 else 0.0
            features["lag_2"] = 1.0 if lag == 2 else 0.0
            features["lag_3plus"] = 1.0 if lag >= 3 else 0.0
            
            # Target
            if use_log_transform:
                features["target"] = np.log1p(b_t_plus_1)
            else:
                features["target"] = b_t_plus_1

            rows.append(features)

    df_train = pd.DataFrame(rows)
    return df_train

# 로그 변환 옵션
USE_LOG_TRANSFORM = False

df_train_model = build_training_data_enhanced(pivot_train, pairs, use_log_transform=USE_LOG_TRANSFORM)
print('생성된 학습 데이터의 shape :', df_train_model.shape)
print('Feature 목록:', [col for col in df_train_model.columns if col != 'target'])


생성된 학습 데이터의 shape : (54154, 17)
Feature 목록: ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag', 'b_t_ma3', 'a_t_lag_ma3', 'b_trend', 'b_trend_2', 'a_trend', 'b_t_scaled', 'a_t_lag_scaled', 'a_t_lag_weighted', 'lag_1', 'lag_2', 'lag_3plus']


## 6. 하이퍼파라미터 탐색 공간 정의


In [6]:
# Feature 선택
feature_cols = [col for col in df_train_model.columns if col != 'target']
train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

print(f"Feature 개수: {len(feature_cols)}")
print(f"학습 샘플 수: {len(train_X)}")

# 하이퍼파라미터 탐색 공간 정의 (Lasso는 alpha만 조정)
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 50.0, 100.0]
}

print("\n하이퍼파라미터 탐색 공간:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")
print(f"\n총 조합 수: {len(param_grid['alpha'])}")

# Lasso는 파라미터가 단순하므로 GridSearchCV 사용
print("\nGridSearchCV로 모든 조합 탐색 예정")


Feature 개수: 16
학습 샘플 수: 54154

하이퍼파라미터 탐색 공간:
  alpha: [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 50.0, 100.0]

총 조합 수: 10

GridSearchCV로 모든 조합 탐색 예정


## 7. 하이퍼파라미터 탐색 수행


In [7]:
# 기본 Lasso 모델
base_model = Lasso(max_iter=2000)  # max_iter 증가 (수렴 보장)

# GridSearchCV로 하이퍼파라미터 탐색
print("하이퍼파라미터 탐색 시작...\n")

grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(train_X, train_y)

print("\n하이퍼파라미터 탐색 완료!")
print(f"최적 파라미터: {grid_search.best_params_}")
print(f"최적 CV 점수 (neg_MAE): {grid_search.best_score_:.6f}")


하이퍼파라미터 탐색 시작...

Fitting 3 folds for each of 10 candidates, totalling 30 fits

하이퍼파라미터 탐색 완료!
최적 파라미터: {'alpha': 100.0}
최적 CV 점수 (neg_MAE): -1449686.987764


## 8. 최적 모델로 예측 함수 정의


In [8]:
def predict_single_model(pivot, pairs, model, feature_cols, target_date, use_log_transform=False):
    """
    단일 모델을 사용한 예측 함수
    """
    # target_date 이전까지의 데이터만 사용
    pivot_for_pred = pivot.loc[:, pivot.columns < target_date].copy()
    
    if len(pivot_for_pred.columns) == 0:
        raise ValueError(f"target_date {target_date} 이전의 데이터가 없습니다.")
    
    months = pivot_for_pred.columns.to_list()
    n_months = len(months)
    
    # 가장 마지막 달 index
    t_last = n_months - 1
    t_prev = t_last - 1 if t_last > 0 else t_last
    t_prev2 = t_last - 2 if t_last >= 2 else 0

    preds = []

    for row in tqdm(pairs.itertuples(index=False), desc="예측 중"):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot_for_pred.index or follower not in pivot_for_pred.index:
            continue

        a_series = pivot_for_pred.loc[leader].values.astype(float)
        b_series = pivot_for_pred.loc[follower].values.astype(float)

        # t_last - lag 가 0 이상인 경우만 예측
        if t_last - lag < 0:
            continue

        # Feature 생성 (학습 시와 동일한 방식)
        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev] if t_prev >= 0 else 0.0
        b_t_2 = b_series[t_prev2] if t_prev2 >= 0 else 0.0
        a_t_lag = a_series[t_last - lag]
        a_t_lag_1 = a_series[t_last - lag - 1] if t_last - lag - 1 >= 0 else 0.0

        # Feature 벡터 생성
        features = {
            "b_t": b_t,
            "b_t_1": b_t_1,
            "a_t_lag": a_t_lag,
            "max_corr": corr,
            "best_lag": float(lag),
            "b_t_ma3": np.mean(b_series[max(0, t_last - 2):t_last + 1]),
            "a_t_lag_ma3": np.mean(a_series[max(0, t_last - lag - 2):t_last - lag + 1]) if t_last - lag >= 0 else 0.0,
            "b_trend": (b_t - b_t_1) / (b_t_1 + 1e-6) if b_t_1 > 0 else 0.0,
            "b_trend_2": (b_t_1 - b_t_2) / (b_t_2 + 1e-6) if b_t_2 > 0 else 0.0,
            "a_trend": (a_t_lag - a_t_lag_1) / (a_t_lag_1 + 1e-6) if a_t_lag_1 > 0 else 0.0,
            "b_t_scaled": b_t / (np.mean(b_series[:t_last+1]) + 1e-6),
            "a_t_lag_scaled": a_t_lag / (np.mean(a_series[:t_last-lag+1]) + 1e-6) if t_last - lag >= 0 else 0.0,
            "a_t_lag_weighted": a_t_lag * abs(corr),
            "lag_1": 1.0 if lag == 1 else 0.0,
            "lag_2": 1.0 if lag == 2 else 0.0,
            "lag_3plus": 1.0 if lag >= 3 else 0.0,
        }
        
        X_test = np.array([[features[col] for col in feature_cols]])

        # 단일 모델 예측
        y_pred = model.predict(X_test)[0]

        # 로그 변환 사용 시 역변환
        if use_log_transform:
            y_pred = np.expm1(y_pred)

        # 후처리: 음수 방지 및 정수 변환
        y_pred = max(0.0, float(y_pred))
        
        # 추가 후처리: 이상치 제한
        if b_t > 0:
            # 현재 값의 20배를 넘지 않도록 제한
            y_pred = min(y_pred, b_t * 20)
            # 최근 트렌드 반영 (선택적)
            if b_t_1 > 0:
                trend = b_t / b_t_1
                y_pred = y_pred * (0.7 + 0.3 * min(trend, 2.0))  # 트렌드 반영하되 과도하지 않게
        
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred


## 9. 정답 데이터 생성 (2025-07-01 실제 값)


In [9]:
# train.csv에서 2025-07-01의 실제 데이터 추출
val_year = val_date.year
val_month = val_date.month

answer_raw = train[
    (train["year"] == val_year) & 
    (train["month"] == val_month)
].copy()

# item_id별 value 합산
answer_monthly = (
    answer_raw
    .groupby("item_id", as_index=False)["value"]
    .sum()
)

# 공행성쌍에 대해 정답 생성
answer_dict = dict(zip(answer_monthly["item_id"], answer_monthly["value"]))

answer_list = []
for row in pairs.itertuples(index=False):
    follower = row.following_item_id
    if follower in answer_dict:
        answer_list.append({
            "leading_item_id": row.leading_item_id,
            "following_item_id": follower,
            "value": answer_dict[follower]
        })
    else:
        answer_list.append({
            "leading_item_id": row.leading_item_id,
            "following_item_id": follower,
            "value": 0
        })

answer_df = pd.DataFrame(answer_list)
print(f"생성된 정답 데이터 수: {len(answer_df)}")
print(f"정답 데이터 value 합계: {answer_df['value'].sum():,.0f}")


생성된 정답 데이터 수: 1453
정답 데이터 value 합계: 7,009,633,460


## 10. 최적 모델로 예측 및 NMAE 평가


In [10]:
# 최적 모델 가져오기
best_model = grid_search.best_estimator_

print("=== 최적 하이퍼파라미터 ===")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

# 최적 모델로 예측
print("\n=== 최적 모델로 예측 수행 ===")
submission = predict_single_model(
    pivot, 
    pairs, 
    best_model, 
    feature_cols, 
    val_date,
    use_log_transform=USE_LOG_TRANSFORM
)

# NMAE 계산
nmae = comovement_nmae(answer_df, submission)
nmae_score = 1 - nmae

print(f"\n=== 검증 결과 ===")
print(f"NMAE: {nmae:.6f}")
print(f"NMAE Score (1 - NMAE): {nmae_score:.6f}")
print(f"\n목표: NMAE < 0.4 (NMAE Score > 0.6)")

print(f"\n예측값 통계:")
print(submission['value'].describe())


=== 최적 하이퍼파라미터 ===
  alpha: 100.0

=== 최적 모델로 예측 수행 ===


예측 중: 1453it [00:00, 3072.06it/s]


=== 검증 결과 ===
NMAE: 0.566999
NMAE Score (1 - NMAE): 0.433001

목표: NMAE < 0.4 (NMAE Score > 0.6)

예측값 통계:
count    1.453000e+03
mean     3.790396e+06
std      1.053090e+07
min      0.000000e+00
25%      2.301590e+05
50%      5.390360e+05
75%      3.153967e+06
max      1.089779e+08
Name: value, dtype: float64


## 11. 하이퍼파라미터 탐색 결과 분석


In [11]:
# 탐색 결과를 DataFrame으로 변환
results_df = pd.DataFrame(grid_search.cv_results_)

# 모든 결과 출력 (alpha 값이 적으므로)
print("=== 모든 하이퍼파라미터 조합 결과 (CV 점수 기준) ===\n")
results_sorted = results_df.sort_values('mean_test_score', ascending=False)[['params', 'mean_test_score', 'std_test_score']]

for idx, row in results_sorted.iterrows():
    rank = list(results_sorted.index).index(idx) + 1
    print(f"순위 {rank}:")
    print(f"  CV 점수 (neg_MAE): {row['mean_test_score']:.6f} (+/- {row['std_test_score']:.6f})")
    print(f"  파라미터: {row['params']}")
    print()

# alpha별 성능 분석
print("\n=== alpha별 성능 분포 ===")
alpha_values = []
scores = []
for idx, row in results_df.iterrows():
    alpha_val = row['params']['alpha']
    alpha_values.append(alpha_val)
    scores.append(row['mean_test_score'])

alpha_df = pd.DataFrame({'alpha': alpha_values, 'score': scores})
alpha_summary = alpha_df.sort_values('alpha')
print(alpha_summary.to_string(index=False))


=== 모든 하이퍼파라미터 조합 결과 (CV 점수 기준) ===

순위 1:
  CV 점수 (neg_MAE): -1449686.987764 (+/- 104892.393985)
  파라미터: {'alpha': 100.0}

순위 2:
  CV 점수 (neg_MAE): -1449732.199826 (+/- 104893.520886)
  파라미터: {'alpha': 50.0}

순위 3:
  CV 점수 (neg_MAE): -1449768.513520 (+/- 104894.425310)
  파라미터: {'alpha': 10.0}

순위 4:
  CV 점수 (neg_MAE): -1449773.055743 (+/- 104894.538380)
  파라미터: {'alpha': 5.0}

순위 5:
  CV 점수 (neg_MAE): -1449775.586978 (+/- 104894.841358)
  파라미터: {'alpha': 2.0}

순위 6:
  CV 점수 (neg_MAE): -1449776.649715 (+/- 104894.744682)
  파라미터: {'alpha': 1.0}

순위 7:
  CV 점수 (neg_MAE): -1449777.163929 (+/- 104894.747365)
  파라미터: {'alpha': 0.5}

순위 8:
  CV 점수 (neg_MAE): -1449777.511659 (+/- 104894.670626)
  파라미터: {'alpha': 0.1}

순위 9:
  CV 점수 (neg_MAE): -1449777.589898 (+/- 104894.653360)
  파라미터: {'alpha': 0.01}

순위 10:
  CV 점수 (neg_MAE): -1449777.597722 (+/- 104894.651633)
  파라미터: {'alpha': 0.001}


=== alpha별 성능 분포 ===
  alpha         score
  0.001 -1.449778e+06
  0.010 -1.449778e+06
  0.100 -1.449778